In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import time
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
data_frame = pd.read_csv('final_dataset_20recipes.csv')

# Drop the name column in the dataframe 
data_frame = data_frame.drop(['name', 'calorie_level', 'recipe_id_mapped'], axis=1)
#print the amount of columns in the dataframe
print("Number of columns: ", len(data_frame.columns))
data_frame = data_frame.groupby('user_id').filter(lambda x: len(x) > 3)

data_frame['user_id'] = data_frame['user_id'].astype('category').cat.codes.values

data_frame['recipe_id'] = data_frame['recipe_id'].astype('category').cat.codes.values
# Print the amount of unique recipes
print("Number of unique recipes: ", data_frame['recipe_id'].nunique())
#print(data_frame.head())
new_column_names = data_frame.columns.values.tolist()
categorical_columns = [
    "technique_0", "technique_1", "technique_2", "technique_3", "technique_4", "technique_5", "technique_6",
    "technique_7", "technique_8", "technique_9", "technique_10", "technique_11", "technique_12", "technique_13",
    "technique_14", "technique_15", "technique_16", "technique_17", "technique_18", "technique_19", "technique_20",
    "technique_21", "technique_22", "technique_23", "technique_24", "technique_25", "technique_26", "technique_27",
    "technique_28", "technique_29", "technique_30", "technique_31", "technique_32", "technique_33", "technique_34",
    "technique_35", "technique_36", "technique_37", "technique_38", "technique_39", "technique_40", "technique_41",
    "technique_42", "technique_43", "technique_44", "technique_45", "technique_46", "technique_47", "technique_48",
    "technique_49", "technique_50", "technique_51", "technique_52", "technique_53", "technique_54", "technique_55",
    "technique_56", "technique_57", "thyme", "half-and-half", "half-and-half cream", "tilapia fillet", "rolled oat",
    "mayonnaise", "potato", "canning salt", "accent seasoning", "whipping cream", "toasted sesame seed", "fennel seed",
    "hamburger bun", "feta cheese", "meatless sauce", "soy sauce", "black bean", "dark beer", "tomato", "paprika",
    "white chocolate bark", "soymilk", "cocoa", "cocoa powder", "spaghetti sauce", "round steak", "lasagna noodle",
    "parmesan cheese", "tomato paste", "all-purpose flmy", "powdered sugar", "parsley", "tomato sauce", "parsley flake",
    "allspice", "carrot", "russet baking potato", "black pepper", "diced tomato", "heavy cream", "top sirloin steak",
    "heavy whipping cream", "cauliflower", "tortilla", "white onion", "lean ground beef", "herb stuffing",
    "cayenne pepper", "white pepper", "aluminum foil", "pastry for single-crust pie", "cayenne", "celery", "pastum",
    "flaked coconut", "celery salt", "milk", "white sugar", "garlic", "milk chocolate chip", "hickory smoke salt",
    "salsa", "flmy", "salt", "hoisin sauce", "salt & pepper", "sprite", "garlic clove", "salt and pepper",
    "angel hair pastum", "lemon", "condensed cream of chicken soup", "honey", "peanut butter", "cheddar",
    "garlic powder", "cheese", "garlic salt", "steak seasoning", "apple", "dressing", "confectioners' sugar",
    "apple cider vinegar", "lemon juice", "boiling water", "stewed tomato", "dried basil", "bone-in ham", "cooked bacon",
    "lemon pepper seasoning", "cooked chicken", "cooked chicken breast", "whole milk", "ginger", "pecan", "sausage",
    "roasting chicken", "whole tomato", "hot chili pepper", "lemon zest", "scallion", "lemon, juice of", "penne",
    "hot coffee", "lemon, zest of", "raisin", "pepper", "fresh blueberry", "bread", "fresh broccoli", "monterey jack cheese",
    "hot mustard", "sea salt", "lentil", "bread flmy", "cooked rice", "hot pepper flake", "lettuce", "breadcrumb",
    "chicken", "hot pepper sauce", "fresh coarse ground black pepper", "seasoned dry bread crumb", "light brown sugar",
    "hot sauce", "chicken breast", "mozzarella", "seasoning salt", "chicken breast half", "hot water", "chicken broth",
    "strong coffee", "unsweetened cocoa", "unsweetened cocoa powder", "ature marshmallow", "picante sauce",
    "chicken drumstick", "fresh ginger", "sugar", "icing sugar", "graham cracker crumb", "broccoli floret", "pie crust",
    "fresh ground black pepper", "fresh ground pepper", "mushroom", "coriander", "piece", "worcestershire sauce", "corn",
    "sesame oil", "sun-dried tomato", "dried oregano", "chicken stock", "dried oregano leaf", "granny smith apple",
    "dried parsley", "dried parsley flake", "yeast", "brown gravy mix", "vanilla", "instant chocolate pudding mix",
    "granulated sugar", "fresh mushroom", "brown sugar", "lime juice", "baby spinach", "chile", "fresh parsley",
    "yellow mustard", "cornstarch", "yellow onion", "dried red pepper flake", "vanilla extract", "chili pepper",
    "chili powder", "sweet butter", "chili sauce", "fresh rosemary", "linguine", "sweet onion", "green bean",
    "green bell pepper", "butter", "shortening", "baking cocoa", "cracked black pepper", "baking powder",
    "red maraschino cherry", "sweet pepper", "baking soda", "red onion", "dried thyme", "balsamic vinegar", "ziti pastum",
    "zucchini", "banana", "red pepper flake", "shrimp", "italian cut green bean", "green chily", "nut",
    "italian plum tomato", "red potato", "nutmeg", "chocolate bark", "dry breadcrumb", "buttermilk", "italian seasoning",
    "oat", "dry crushed red pepper", "locatelli cheese", "green maraschino cherry", "chocolate chip", "chocolate candy",
    "barbecue sauce", "green pepper", "jalapeno pepper", "cream cheese", "red wine", "cream", "swiss cheese",
    "red wine vinegar", "oil", "cream of chicken soup", "dry mustard", "basil", "cream of mushroom soup", "refried bean",
    "cabbage", "vegetable broth", "old bay seasoning", "cream of tartar", "vegetable oil", "syrup", "cajun seasoning",
    "bay leaf", "refrigerated crescent dinner roll", "olive oil", "table salt", "taco cheese", "table cream", "onion",
    "creamy peanut butter", "taco seasoning", "cake mix", "chocolate milk", "plum tomato", "macaroni",
    "small white bean", "small shell pastum", "ground beef", "ground black pepper", "onion powder", "ground cinnamon",
    "ground cumin", "beef broth", "kalamata olive", "kale", "ground ginger", "vermicelli", "pork back rib",
    "crushed pineapple", "vidalia onion", "crushed red pepper flake", "egg", "vinegar", "ricotta cheese", "crusty bread",
    "ground nutmeg", "pork chop", "chopped tomato", "rice wine vinegar", "ground pepper", "ground red pepper", "ketchup",
    "elbow macaroni", "pork loin chop", "beef roast", "cumin", "walnut", "maraschino cherry", "margarine", "pork rib",
    "beef sirloin", "kidney bean", "texas toast thick bread", "english cucumber", "cinnamon", "water", "water chestnut",
    "oregano", "oreo cookies", "thin spaghettus", "dark brown sugar", "sour cream"]

# Define columns to be scaled
numerical_columns = ['calories', 'minutes', 'n_steps', 'n_ingredients', 'percent_fat', 'percent_sugar', 
                     'percent_sodium', 'percent_protein', 'percent_sat_fat', 'percent_carb']

numerical_transformer = StandardScaler()

# Apply the transform to the columns in the dataframe that are in the numerical_columns list
data_frame[numerical_columns] = numerical_transformer.fit_transform(data_frame[numerical_columns])

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Assuming that the last column is the target
        data = torch.tensor(self.dataframe.iloc[idx, :-1].values, dtype=torch.float32)
        target = torch.tensor(self.dataframe.iloc[idx, -1], dtype=torch.float32)

        return data, target

# Create an instance of the custom dataset
dataset = CustomDataset(data_frame)



Number of columns:  185
Number of unique recipes:  19
